In [1]:
### inputs ###
task = "rest" # rest or bart

In [2]:
import os
import openpyxl
import nibabel as nib
import numpy as np
import pandas as pd
import subprocess
import scipy.io

In [3]:
wb = openpyxl.load_workbook('Subjects1.xlsx')
wb.get_sheet_names()
sheet = wb.get_sheet_by_name('Sheet1')
N = sheet.max_row

c:\users\paria\appdata\local\programs\python\python36\lib\site-packages\openpyxl\reader\worksheet.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
Data_DIR = "F://UCLA//func//"
os.chdir(Data_DIR)
os.getcwd()

'F:\\UCLA\\func'

In [8]:
## create the 36 parameter file
for i in range(1,N):
    ID = sheet.cell(row=i+1, column=1).value
    sub = str(ID) 
    tsv_file = "sub-" + sub+ "_task-" + task + "_bold_confounds.tsv"
    df = pd.read_table(tsv_file, sep='\t', header=0)
    [dim1, dim2] = np.shape(df)
    conf_clmns = [row for row in df if row == "WhiteMatter" or row == "X" or row == "Y" or row == "Z" or row == "RotX" or row == "RotY" or row == "RotZ"]
    data_confs = np.zeros((dim1, 4*7))
    cntr = 0
    for item in conf_clmns:
        data_confs[:, cntr] = np.array(df[item])
        data_confs[:, cntr+7] = np.gradient(data_confs[:, cntr])
        data_confs[:, cntr+2*7] = np.gradient(data_confs[:, cntr+8])
        data_confs[:, cntr+3*7] = np.square(data_confs[:, cntr+2*8])
        cntr = cntr + 1
    data_confs = data_confs[5:,:]
    # set the name of the desired data set here:
    conf_name = "sub-" + sub+ "_task-" + task + "_bold_36p_confounds.mat"
    scipy.io.savemat(conf_name, {'mydata': data_confs})